In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import re
import warnings
from matplotlib.ticker import AutoMinorLocator
import matplotlib as mpl

In [6]:
# Configure visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Create plot directory
PLOTS_DIR = "plot_results"
RESULTS_DIR = "results"
os.makedirs(PLOTS_DIR, exist_ok=True)

# Ignore warnings
warnings.filterwarnings('ignore')

In [11]:
def load_experiment_data():
    """
    Load all experiment data from the results directory.
    Handles both regular and partial file patterns.
    
    Returns:
    --------
    dict: Structured dictionary containing all experiment data
    """
    print("Loading experiment data...")
    
    experiments = {}
    error_types = [d for d in os.listdir(RESULTS_DIR) 
                  if os.path.isdir(os.path.join(RESULTS_DIR, d)) and not d.startswith('.')]
    
    for error_type in error_types:
        error_dir = os.path.join(RESULTS_DIR, error_type)
        if not os.path.isdir(error_dir):
            continue
            
        print(f"Processing {error_type} error type...")
        
        # Process all files in this error type directory
        all_files = os.listdir(error_dir)
        
        # Group files by experiment (excluding partial files)
        experiment_files = {}
        for file in all_files:
            # Skip partial files and metadata files for grouping
            if file.endswith('.csv') and '_partial_' not in file and '_metadata.csv' not in file:
                experiment_files[file] = []
                
        # Find metadata files for each experiment
        for exp_file in experiment_files:
            base_name = exp_file.replace('.csv', '')
            metadata_file = f"{base_name}_metadata.csv"
            if metadata_file in all_files:
                experiment_files[exp_file].append(metadata_file)
        
        # Load each experiment
        for exp_file, related_files in experiment_files.items():
            # Parse experiment details from filename
            parts = exp_file.replace('.csv', '').split('_')
            
            # Extract initial state (first part)
            initial_state = parts[0]
            
            # Extract QEC method (everything until parameter name)
            qec_method = '_'.join(parts[1:-2]) if len(parts) > 3 else parts[1]
            
            # Extract error parameters
            param_name = parts[-2]
            param_value = parts[-1]
            #error_params = {param_name: float(param_value)}


            if param_name == "axis" and error_type == "coherent_overrotation":
                error_params = {param_name: param_value}  # Keep as string
            else:
                try:
                    error_params = {param_name: float(param_value)}
                except ValueError:
                    # For any other cases where conversion fails
                    error_params = {param_name: param_value}
            
            # For mixed_xz error, the parameters might be structured differently
            if error_type == "mixed_xz" and len(parts) >= 5:
                try:
                    error_params = {
                        parts[-4]: float(parts[-3]),
                        parts[-2]: float(parts[-1])
                    }
                except:
                    # Fallback to simpler parsing if the above fails
                    error_params = {param_name: float(param_value)}
            
            # Create experiment key
            exp_key = f"{initial_state}_{error_type}_{qec_method}_{str(error_params)}"
            
            # Load the main data file
            file_path = os.path.join(error_dir, exp_file)
            try:
                data = pd.read_csv(file_path)
                
                # Load metadata if available
                metadata = None
                if len(related_files) > 0:
                    metadata_path = os.path.join(error_dir, related_files[0])
                    metadata = pd.read_csv(metadata_path)
                
                # Store experiment data
                experiments[exp_key] = {
                    'initial_state': initial_state,
                    'error_type': error_type,
                    'qec_method': qec_method,
                    'error_params': error_params,
                    'data': data,
                    'metadata': metadata
                }
            except Exception as e:
                print(f"Error loading {file_path}: {str(e)}")
    
    print(f"Loaded {len(experiments)} experiments")
    return experiments

In [12]:
def analyze_by_initial_state(experiments):
    """
    Analyze how different initial states respond to various error types and QEC methods.
    Creates visualizations and returns state-specific analysis.
    """
    print("\nAnalyzing performance by initial state...")
    
    # Group experiments by initial state
    state_results = defaultdict(list)
    
    for exp_key, exp_data in experiments.items():
        state = exp_data['initial_state']
        state_results[state].append(exp_data)
    
    # Create plots directory for state analysis
    state_plots_dir = os.path.join(PLOTS_DIR, "state_analysis")
    os.makedirs(state_plots_dir, exist_ok=True)
    
    state_summaries = {}
    
    # Analyze each state
    for state, exps in state_results.items():
        print(f"Analyzing state: {state}")
        
        # Group by error type for this state
        error_type_results = defaultdict(list)
        for exp in exps:
            error_type_results[exp['error_type']].append(exp)
        
        # Create plots for each error type
        for error_type, error_exps in error_type_results.items():
            # Prepare data for plotting
            qec_methods = []
            avg_fidelities = []
            std_fidelities = []
            avg_no_qec = []
            std_no_qec = []
            error_params = []
            
            # Group by QEC method and error parameter
            for exp in error_exps:
                qec_method = exp['qec_method']
                if qec_method == 'none':
                    continue
                
                # Get the error parameter (key-value pair as string)
                param_str = ", ".join([f"{k}={v}" for k, v in exp['error_params'].items()])
                
                # Calculate metrics from data
                avg_fidelity = exp['data']['fidelity_with_qec'].mean()
                std_fidelity = exp['data']['fidelity_with_qec'].std()
                
                avg_fidelity_no_qec = exp['data']['fidelity_without_qec'].mean()
                std_fidelity_no_qec = exp['data']['fidelity_without_qec'].std()
                
                qec_methods.append(qec_method)
                avg_fidelities.append(avg_fidelity)
                std_fidelities.append(std_fidelity)
                avg_no_qec.append(avg_fidelity_no_qec)
                std_no_qec.append(std_fidelity_no_qec)
                error_params.append(param_str)
            
            # Plot the results if we have data
            if qec_methods:
                plt.figure(figsize=(14, 8))
                
                # Create positions for grouped bars
                x = np.arange(len(error_params))
                width = 0.15
                n_methods = len(set(qec_methods))
                offsets = np.linspace(-((n_methods-1)/2)*width, ((n_methods-1)/2)*width, n_methods)
                
                # Create a mapping of QEC methods to colors
                method_colors = {
                    'three_qubit_bit_flip': '#1f77b4',
                    'three_qubit_phase_flip': '#ff7f0e',
                    'five_qubit': '#2ca02c',
                    'shor_nine': '#d62728',
                    'steane_seven': '#9467bd'
                }
                
                # Plot each QEC method
                for i, method in enumerate(set(qec_methods)):
                    # Find indices for this method
                    indices = [j for j, m in enumerate(qec_methods) if m == method]
                    
                    # Extract data for this method
                    method_params = [error_params[j] for j in indices]
                    method_fidelity = [avg_fidelities[j] for j in indices]
                    method_std = [std_fidelities[j] for j in indices]
                    
                    # Get positions for this method's bars
                    #positions = x + offsets[i]

                    #Use only positions corresponding to available data:
                    positions = [x[error_params.index(param)] + offsets[i] for param in method_params]
                    
                    # Plot with error bars
                    color = method_colors.get(method, f'C{i}')
                    plt.bar(positions, method_fidelity, width, yerr=method_std, 
                            label=method, color=color, alpha=0.7)
                
                # Plot no QEC results
                plt.bar(x + offsets[-1] + width, avg_no_qec, width, yerr=std_no_qec,
                        label='No QEC', color='gray', alpha=0.7)
                
                # Configure plot
                plt.xlabel('Error Parameters')
                plt.ylabel('Average Fidelity')
                plt.title(f'QEC Performance for {state} State with {error_type} Error')
                plt.xticks(x, error_params, rotation=45)
                plt.ylim(0, 1.1)
                plt.legend(title='QEC Method')
                plt.tight_layout()
                
                # Save plot
                plot_file = f'{state}_{error_type}_qec_comparison.png'
                plt.savefig(os.path.join(state_plots_dir, plot_file))
                plt.close()
        
        # Create overall state vulnerability analysis
        state_vulnerability = {}
        for exp in exps:
            error_type = exp['error_type']
            qec_method = exp['qec_method']
            
            if qec_method != 'none':
                continue
                
            # Average fidelity across all error parameters for this error type
            avg_fidelity = exp['data']['fidelity_without_qec'].mean()
            
            if error_type not in state_vulnerability:
                state_vulnerability[error_type] = []
            
            state_vulnerability[error_type].append(avg_fidelity)
        
        # Average vulnerability across parameter values
        vulnerability_scores = {}
        for error_type, fidelities in state_vulnerability.items():
            vulnerability_scores[error_type] = np.mean(fidelities)
        
        # Create vulnerability plot
        if vulnerability_scores:
            plt.figure(figsize=(10, 6))
            error_types = list(vulnerability_scores.keys())
            vulnerabilities = [1 - vulnerability_scores[et] for et in error_types]
            
            plt.bar(error_types, vulnerabilities, color='crimson', alpha=0.7)
            plt.xlabel('Error Type')
            plt.ylabel('Vulnerability (1 - Fidelity)')
            plt.title(f'Error Vulnerability for {state} State')
            plt.xticks(rotation=45)
            plt.ylim(0, 1)
            plt.tight_layout()
            
            # Save plot
            plot_file = f'{state}_error_vulnerability.png'
            plt.savefig(os.path.join(state_plots_dir, plot_file))
            plt.close()
            
            # Store summary
            state_summaries[state] = {
                'vulnerability_scores': vulnerability_scores,
                'best_qec_methods': {}  # Will fill this in later
            }
    
    return state_summaries

In [5]:
def analyze_by_qec_method(experiments):
    """
    Analyze the effectiveness of different QEC methods across error types.
    Creates visualizations and returns QEC-specific analysis.
    """
    print("\nAnalyzing performance by QEC method...")
    
    # Create plots directory for QEC analysis
    qec_plots_dir = os.path.join(PLOTS_DIR, "qec_analysis")
    os.makedirs(qec_plots_dir, exist_ok=True)
    
    # Group experiments by QEC method
    qec_results = defaultdict(list)
    
    for exp_key, exp_data in experiments.items():
        qec_method = exp_data['qec_method']
        qec_results[qec_method].append(exp_data)
    
    qec_summaries = {}
    
    # Analyze each QEC method
    for qec_method, exps in qec_results.items():
        if qec_method == 'none':
            continue
            
        print(f"Analyzing QEC method: {qec_method}")
        
        # Group by error type
        error_type_results = defaultdict(list)
        for exp in exps:
            error_type_results[exp['error_type']].append(exp)
        
        # Analyze effectiveness against different error types
        effectiveness_by_error = {}
        
        for error_type, error_exps in error_type_results.items():
            # Average error mitigation factors
            error_mitigation_factors = []
            error_params = []
            
            for exp in error_exps:
                # Skip if no data
                if exp['data'] is None or exp['data'].empty:
                    continue
                
                # Get the error parameter (key-value pair as string)
                param_str = ", ".join([f"{k}={v}" for k, v in exp['error_params'].items()])
                
                # Calculate average error mitigation factor
                avg_emf = exp['data']['error_mitigation_factor'].mean()
                error_mitigation_factors.append(avg_emf)
                error_params.append(param_str)
            
            if error_mitigation_factors:
                effectiveness_by_error[error_type] = {
                    'avg_emf': np.mean(error_mitigation_factors),
                    'error_mitigation_factors': error_mitigation_factors,
                    'error_params': error_params
                }
        
        # Plot effectiveness against different error types
        if effectiveness_by_error:
            plt.figure(figsize=(12, 7))
            error_types = list(effectiveness_by_error.keys())
            avg_emfs = [effectiveness_by_error[et]['avg_emf'] for et in error_types]
            
            plt.bar(error_types, avg_emfs, color='teal', alpha=0.7)
            plt.axhline(y=1.0, color='red', linestyle='--', label='No Improvement')
            plt.xlabel('Error Type')
            plt.ylabel('Error Mitigation Factor')
            plt.title(f'Effectiveness of {qec_method} QEC Method by Error Type')
            plt.xticks(rotation=45)
            plt.legend()
            plt.tight_layout()
            
            # Save plot
            plot_file = f'{qec_method}_error_effectiveness.png'
            plt.savefig(os.path.join(qec_plots_dir, plot_file))
            plt.close()
        
        # Plot detailed error mitigation factors for each error type
        for error_type, data in effectiveness_by_error.items():
            if not data['error_params']:
                continue
                
            plt.figure(figsize=(10, 6))
            x = np.arange(len(data['error_params']))
            plt.bar(x, data['error_mitigation_factors'], color='slateblue', alpha=0.7)
            plt.axhline(y=1.0, color='red', linestyle='--', label='No Improvement')
            plt.xlabel('Error Parameters')
            plt.ylabel('Error Mitigation Factor')
            plt.title(f'{qec_method} against {error_type} Error')
            plt.xticks(x, data['error_params'], rotation=45)
            plt.legend()
            plt.tight_layout()
            
            # Save plot
            plot_file = f'{qec_method}_{error_type}_detailed.png'
            plt.savefig(os.path.join(qec_plots_dir, plot_file))
            plt.close()
        
        # Store summary
        qec_summaries[qec_method] = effectiveness_by_error
    
    return qec_summaries

In [6]:
def generate_error_type_analysis(experiments):
    """
    Analyze how different error types affect qubit states and QEC performance.
    """
    print("\nAnalyzing performance by error type...")
    
    # Create plots directory for error analysis
    error_plots_dir = os.path.join(PLOTS_DIR, "error_analysis")
    os.makedirs(error_plots_dir, exist_ok=True)
    
    # Group experiments by error type
    error_results = defaultdict(list)
    
    for exp_key, exp_data in experiments.items():
        error_type = exp_data['error_type']
        error_results[error_type].append(exp_data)
    
    error_summaries = {}
    
    # Analyze each error type
    for error_type, exps in error_results.items():
        print(f"Analyzing error type: {error_type}")
        
        # Group by QEC method
        qec_method_results = defaultdict(list)
        for exp in exps:
            qec_method_results[exp['qec_method']].append(exp)
        
        # Compare QEC methods for this error type
        qec_method_effectiveness = {}
        
        for qec_method, qec_exps in qec_method_results.items():
            if qec_method == 'none':
                continue
                
            # Average across all error parameters and states
            avg_mitigation_factors = []
            
            for exp in qec_exps:
                if exp['data'] is not None and not exp['data'].empty:
                    avg_emf = exp['data']['error_mitigation_factor'].mean()
                    avg_mitigation_factors.append(avg_emf)
            
            if avg_mitigation_factors:
                qec_method_effectiveness[qec_method] = np.mean(avg_mitigation_factors)
        
        # Plot QEC method comparison for this error type
        if qec_method_effectiveness:
            plt.figure(figsize=(10, 6))
            methods = list(qec_method_effectiveness.keys())
            effectiveness = list(qec_method_effectiveness.values())
            
            # Sort by effectiveness
            sorted_indices = np.argsort(effectiveness)[::-1]  # Descending
            methods = [methods[i] for i in sorted_indices]
            effectiveness = [effectiveness[i] for i in sorted_indices]
            
            plt.bar(methods, effectiveness, color='darkgreen', alpha=0.7)
            plt.axhline(y=1.0, color='red', linestyle='--', label='No Improvement')
            plt.xlabel('QEC Method')
            plt.ylabel('Average Error Mitigation Factor')
            plt.title(f'QEC Method Effectiveness for {error_type} Error')
            plt.xticks(rotation=45)
            plt.legend()
            plt.tight_layout()
            
            # Save plot
            plot_file = f'{error_type}_qec_comparison.png'
            plt.savefig(os.path.join(error_plots_dir, plot_file))
            plt.close()
        
        # Analyze effect of error parameters for this error type
        # Group by error parameter value
        param_results = defaultdict(list)
        
        # Extract parameter names
        param_names = set()
        for exp in exps:
            param_names.update(exp['error_params'].keys())
        
        # For each parameter, analyze effect on fidelity
        for param_name in param_names:
            param_data = []
            
            for exp in exps:
                if param_name in exp['error_params'] and exp['qec_method'] == 'none':
                    param_value = exp['error_params'][param_name]
                    avg_fidelity = exp['data']['fidelity_without_qec'].mean()
                    param_data.append((param_value, avg_fidelity))
            
            if param_data:
                # Sort by parameter value
                param_data.sort(key=lambda x: x[0])
                param_values = [p[0] for p in param_data]
                fidelities = [p[1] for p in param_data]
                
                plt.figure(figsize=(10, 6))
                plt.plot(param_values, fidelities, 'o-', color='darkblue', linewidth=2)
                plt.xlabel(f'Error Parameter: {param_name}')
                plt.ylabel('Average Fidelity (No QEC)')
                plt.title(f'Effect of {param_name} on Fidelity for {error_type} Error')
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.tight_layout()
                
                # Save plot
                plot_file = f'{error_type}_{param_name}_effect.png'
                plt.savefig(os.path.join(error_plots_dir, plot_file))
                plt.close()
        
        # Store summary
        error_summaries[error_type] = {
            'best_qec_methods': qec_method_effectiveness
        }
    
    return error_summaries


In [7]:
def generate_comprehensive_report(experiments, state_summaries, qec_summaries, error_summaries):
    """
    Generate a comprehensive report of the analyses.
    """
    print("\nGenerating comprehensive report...")
    
    # Create report directory
    report_dir = os.path.join(PLOTS_DIR, "report")
    os.makedirs(report_dir, exist_ok=True)
    
    # Prepare report data
    report = {
        'state_vulnerability': {},
        'best_qec_by_error': {},
        'error_correctability': {}
    }
    
    # State vulnerability
    for state, summary in state_summaries.items():
        if 'vulnerability_scores' in summary:
            report['state_vulnerability'][state] = {
                'most_vulnerable_to': min(summary['vulnerability_scores'], 
                                         key=summary['vulnerability_scores'].get),
                'least_vulnerable_to': max(summary['vulnerability_scores'], 
                                          key=summary['vulnerability_scores'].get),
                'average_vulnerability': 1 - np.mean(list(summary['vulnerability_scores'].values()))
            }
    
    # Best QEC by error type
    for error_type, summary in error_summaries.items():
        if 'best_qec_methods' in summary and summary['best_qec_methods']:
            best_method = max(summary['best_qec_methods'], 
                             key=summary['best_qec_methods'].get)
            best_emf = summary['best_qec_methods'][best_method]
            
            report['best_qec_by_error'][error_type] = {
                'best_method': best_method,
                'error_mitigation_factor': best_emf
            }
    
    # Error correctability
    for error_type, summary in error_summaries.items():
        if 'best_qec_methods' in summary and summary['best_qec_methods']:
            best_emf = max(summary['best_qec_methods'].values())
            report['error_correctability'][error_type] = best_emf
    
    # Create summary plots
    
    # State vulnerability comparison
    if report['state_vulnerability']:
        plt.figure(figsize=(12, 8))
        states = list(report['state_vulnerability'].keys())
        vulnerabilities = [report['state_vulnerability'][s]['average_vulnerability'] for s in states]
        
        # Sort by vulnerability
        sorted_indices = np.argsort(vulnerabilities)[::-1]  # Descending
        states = [states[i] for i in sorted_indices]
        vulnerabilities = [vulnerabilities[i] for i in sorted_indices]
        
        plt.bar(states, vulnerabilities, color='firebrick', alpha=0.7)
        plt.xlabel('Initial State')
        plt.ylabel('Average Vulnerability (1 - Fidelity)')
        plt.title('Vulnerability of Different Quantum States to Errors')
        plt.ylim(0, 1)
        plt.tight_layout()
        
        # Save plot
        plt.savefig(os.path.join(report_dir, 'state_vulnerability_comparison.png'))
        plt.close()
    
    # Error correctability comparison
    if report['error_correctability']:
        plt.figure(figsize=(12, 8))
        error_types = list(report['error_correctability'].keys())
        correctability = list(report['error_correctability'].values())
        
        # Sort by correctability
        sorted_indices = np.argsort(correctability)[::-1]  # Descending
        error_types = [error_types[i] for i in sorted_indices]
        correctability = [correctability[i] for i in sorted_indices]
        
        plt.bar(error_types, correctability, color='darkgreen', alpha=0.7)
        plt.axhline(y=1.0, color='red', linestyle='--', label='No Improvement')
        plt.xlabel('Error Type')
        plt.ylabel('Best Error Mitigation Factor')
        plt.title('Correctability of Different Error Types')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        
        # Save plot
        plt.savefig(os.path.join(report_dir, 'error_correctability_comparison.png'))
        plt.close()
    
    # Best QEC method by error type
    if report['best_qec_by_error']:
        plt.figure(figsize=(12, 8))
        error_types = list(report['best_qec_by_error'].keys())
        best_methods = [report['best_qec_by_error'][et]['best_method'] for et in error_types]
        emfs = [report['best_qec_by_error'][et]['error_mitigation_factor'] for et in error_types]
        
        # Sort by EMF
        sorted_indices = np.argsort(emfs)[::-1]  # Descending
        error_types = [error_types[i] for i in sorted_indices]
        best_methods = [best_methods[i] for i in sorted_indices]
        emfs = [emfs[i] for i in sorted_indices]
        
        # Create colormap for methods
        method_colors = {
            'three_qubit_bit_flip': '#1f77b4',
            'three_qubit_phase_flip': '#ff7f0e',
            'five_qubit': '#2ca02c',
            'shor_nine': '#d62728',
            'steane_seven': '#9467bd'
        }
        colors = [method_colors.get(method, 'gray') for method in best_methods]
        
        plt.figure(figsize=(14, 8))
        bars = plt.bar(error_types, emfs, color=colors, alpha=0.7)
        
        # Add method labels to the bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    best_methods[i],
                    ha='center', va='bottom', rotation=90, fontsize=10)
        
        plt.axhline(y=1.0, color='red', linestyle='--', label='No Improvement')
        plt.xlabel('Error Type')
        plt.ylabel('Error Mitigation Factor')
        plt.title('Best QEC Method by Error Type')
        plt.xticks(rotation=45)
        plt.legend(handles=[plt.Line2D([0], [0], color='red', linestyle='--', label='No Improvement')])
        plt.tight_layout()
        
        # Save plot
        plt.savefig(os.path.join(report_dir, 'best_qec_by_error.png'))
        plt.close()
    
    # Save report data as JSON
    import json
    with open(os.path.join(report_dir, 'analysis_report.json'), 'w') as f:
        json.dump(report, f, indent=4)
    
    print("Comprehensive report generated successfully!")
    return report


In [7]:
def configure_plots():
    """Configure matplotlib rcParams for publication-quality plots"""
    plt.style.use('seaborn-v0_8-whitegrid')
    mpl.rcParams.update({
        'font.family': 'serif',
        'font.size': 12,
        'axes.labelsize': 14,
        'axes.titlesize': 16,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'legend.fontsize': 10,
        'legend.frameon': True,
        'legend.framealpha': 0.8,
        'figure.figsize': (10, 6),
        'figure.dpi': 300,
        'text.usetex': False,  # Set True if LaTeX is installed
        'mathtext.fontset': 'stix',
        'axes.prop_cycle': plt.cycler('color', plt.cm.cubehelix(np.linspace(0,1,6))),
        'axes.axisbelow': True
    })

In [15]:
def create_scatter_plots(experiments):
    """Create professional scatter plots for all analysis requirements"""
    print("\nCreating professional scatter plots...")
    configure_plots()
    
    # Create plot directories
    plot_dirs = {
        'state_error': os.path.join(PLOTS_DIR, "state_error_scatters"),
        'state_qec': os.path.join(PLOTS_DIR, "state_qec_scatters"),
        'state_fidelity': os.path.join(PLOTS_DIR, "state_fidelity_scatters")
    }
    for d in plot_dirs.values():
        os.makedirs(d, exist_ok=True)

    # Get unique states, error types, and QEC methods
    states = sorted({exp['initial_state'] for exp in experiments.values()})
    error_types = sorted({exp['error_type'] for exp in experiments.values()})
    qec_methods = sorted({exp['qec_method'] for exp in experiments.values() if exp['qec_method'] != 'none'})

    # 1. Per-state fidelity scatter plots
    print("Creating state-wise fidelity distributions...")
    for state in states:
        fig, ax = plt.subplots()
        state_data = [exp for exp in experiments.values() if exp['initial_state'] == state]
        
        # Create colormap based on fidelity values
        fidelities = np.concatenate([exp['data']['fidelity_with_qec'] for exp in state_data])
        norm = mpl.colors.Normalize(vmin=0, vmax=1)
        cmap = plt.cm.viridis
        
        # Plot each experiment's data
        for exp in state_data:
            label = f"{exp['error_type']} ({exp['qec_method']})"
            sc = ax.scatter(
                x=exp['data'].index,
                y=exp['data']['fidelity_with_qec'],
                c=exp['data']['fidelity_with_qec'],
                cmap=cmap,
                norm=norm,
                alpha=0.7,
                edgecolor='w',
                linewidth=0.3,
                label=label
            )
        
        # Format plot
        ax.set_title(f"Fidelity Distribution for {state} State", pad=20)
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fidelity")
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        
        # Add colorbar
        cbar = fig.colorbar(sc, ax=ax)
        cbar.set_label("Fidelity Value")
        
        # Save and close
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dirs['state_fidelity'], f"{state}_fidelity_scatter.png"))
        plt.close()

    # 2. State vs Error Type plots
    print("Creating state vs error type analysis...")
    for state in states:
        fig, axs = plt.subplots(3, 3, figsize=(15, 15))
        fig.suptitle(f"Fidelity by Error Type: {state} State", y=0.92)
        
        for idx, error_type in enumerate(error_types):
            ax = axs.flatten()[idx]
            exp_data = [exp for exp in experiments.values() 
                       if exp['initial_state'] == state 
                       and exp['error_type'] == error_type]
            
            # Create violin plot distribution
            data = np.concatenate([exp['data']['fidelity_with_qec'] for exp in exp_data])
            parts = ax.violinplot(data, showmeans=True, showextrema=False)
            
            # Style violin plots
            for pc in parts['bodies']:
                pc.set_facecolor(plt.cm.tab10(idx))
                pc.set_edgecolor('black')
                pc.set_alpha(0.8)
            
            # Format subplot
            ax.set_title(error_type.replace('_', ' ').title())
            ax.set_ylim(0, 1.1)
            ax.yaxis.set_minor_locator(AutoMinorLocator())
            ax.grid(True, which='both', axis='y', alpha=0.5)
        
        # Remove empty subplots
        for idx in range(len(error_types), len(axs.flatten())):
            axs.flatten()[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dirs['state_error'], f"{state}_error_comparison.png"))
        plt.close()

    # 3. State vs QEC Method plots
    print("Creating state vs QEC method analysis...")
    for state in states:
        fig = plt.figure(figsize=(12, 8))
        gs = fig.add_gridspec(2, 2, width_ratios=[3,1], height_ratios=[1,3])
        
        main_ax = fig.add_subplot(gs[1,0])
        hist_ax = fig.add_subplot(gs[1,1], sharey=main_ax)
        cbar_ax = fig.add_subplot(gs[0,:])
        
        # Main scatter plot
        for method_idx, method in enumerate(qec_methods):
            exp_data = [exp for exp in experiments.values() 
                       if exp['initial_state'] == state 
                       and exp['qec_method'] == method]
            
            if not exp_data:
                continue
            
            # Aggregate data
            fidelities = np.concatenate([exp['data']['fidelity_with_qec'] for exp in exp_data])
            params = [str(exp['error_params']) for exp in exp_data for _ in exp['data']['fidelity_with_qec']]
            
            # Create numeric mapping for parameters
            unique_params = sorted(set(params))
            param_map = {p:i for i,p in enumerate(unique_params)}
            param_ids = [param_map[p] for p in params]
            
            # Plot
            colors = [method_idx] * len(fidelities)
            sc = main_ax.scatter(
                x=param_ids,
                y=fidelities,
                c=colors,
                cmap='tab20',
                vmin=0,
                vmax=len(qec_methods)-1,
                alpha=0.7,
                edgecolor='w',
                linewidth=0.3,
                label=method
            )
        
        # Format main plot
        main_ax.set_xticks(range(len(unique_params)))
        main_ax.set_xticklabels(unique_params, rotation=45, ha='right')
        main_ax.set_xlabel("Error Parameters")
        main_ax.set_ylabel("Fidelity")
        main_ax.set_ylim(0, 1.1)
        
        # Add histogram
        all_fidelities = [exp['data']['fidelity_with_qec'] for exp in experiments.values() 
                         if exp['initial_state'] == state]
        hist_ax.hist(np.concatenate(all_fidelities), orientation='horizontal', 
                    bins=20, density=True, alpha=0.7)
        hist_ax.set_xlabel("Density")
        
        # Add colorbar
        cbar = fig.colorbar(sc, cax=cbar_ax, orientation='horizontal')
        cbar.set_label("QEC Methods")
        cbar.set_ticks(range(len(qec_methods)))
        cbar.set_ticklabels(qec_methods)
        
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dirs['state_qec'], f"{state}_qec_analysis.png"))
        plt.close()

    # 4. Average fidelity plot
    print("Creating average fidelity comparison...")
    state_avg = {}
    for state in states:
        state_data = [exp['data']['fidelity_with_qec'] for exp in experiments.values() 
                     if exp['initial_state'] == state]
        state_avg[state] = (np.mean(np.concatenate(state_data)),
                           np.std(np.concatenate(state_data)))

    fig, ax = plt.subplots()
    ordered_states = sorted(state_avg.keys(), key=lambda x: state_avg[x][0], reverse=True)
    x_pos = np.arange(len(ordered_states))
    
    # Create bars with error caps
    bars = ax.bar(x_pos, [state_avg[s][0] for s in ordered_states],
                 yerr=[state_avg[s][1] for s in ordered_states],
                 capsize=5, alpha=0.7,
                 color=plt.cm.cubehelix(np.linspace(0,1,len(ordered_states))))
    
    # Add value annotations
    for bar, (avg, std) in zip(bars, [state_avg[s] for s in ordered_states]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height+0.02,
               f'{avg:.2f} ± {std:.2f}',
               ha='center', va='bottom')
    
    # Format plot
    ax.set_title("Average Fidelity Across All Experiments")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(ordered_states)
    ax.set_ylabel("Fidelity")
    ax.set_ylim(0, 1.1)
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    plt.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "average_fidelity_comparison.png"))
    plt.close()

    print("All professional plots generated successfully!")

In [18]:
def create_state_error_plots(experiments):
    """Create improved violin plots for each state and error type"""
    print("Creating improved state vs error type analysis...")
    
    plot_dir = os.path.join(PLOTS_DIR, "state_error_scatters")
    os.makedirs(plot_dir, exist_ok=True)
    
    states = sorted({exp['initial_state'] for exp in experiments.values()})
    error_types = sorted({exp['error_type'] for exp in experiments.values()})
    
    for state in states:
        # Increase figure size for better spacing
        fig, axs = plt.subplots(3, 3, figsize=(15, 16))
        
        # Add title with proper spacing
        fig.suptitle(f"Fidelity by Error Type: {state} State", 
                    fontsize=20, fontweight='bold', y=0.98)
        # Add extra space at the top
        plt.subplots_adjust(top=0.92)
        
        # Create violin plots for each error type
        for idx, error_type in enumerate(error_types):
            if idx >= len(axs.flatten()):
                break
                
            ax = axs.flatten()[idx]
            exp_data = [exp for exp in experiments.values() 
                       if exp['initial_state'] == state 
                       and exp['error_type'] == error_type]
            
            if not exp_data:
                ax.text(0.5, 0.5, 'No Data', ha='center', va='center')
                ax.set_title(error_type.replace('_', ' ').title())
                continue
            
            # Add data and styling
            data = np.concatenate([exp['data']['fidelity_with_qec'] for exp in exp_data])
            parts = ax.violinplot(data, showmeans=False, showextrema=True)
            
            # Add mean and median markers
            ax.scatter(1, np.mean(data), color='white', s=30, zorder=3, edgecolor='black')
            ax.hlines(np.median(data), 0.8, 1.2, color='k', linestyle='-', lw=2)
            
            for pc in parts['bodies']:
                pc.set_facecolor(plt.cm.tab10(idx))
                pc.set_edgecolor('black')
                pc.set_alpha(0.7)
            
            # Format subplot
            ax.set_title(error_type.replace('_', ' ').title(), fontsize=14)
            ax.set_ylim(0, 1.1)
            ax.set_xticks([])
            ax.set_ylabel("Fidelity")
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))
            ax.grid(True, which='both', axis='y', alpha=0.5, linestyle=':')
        
        # Turn off empty subplots
        for idx in range(len(error_types), len(axs.flatten())):
            axs.flatten()[idx].axis('off')
        
        plt.tight_layout(rect=[0, 0, 1, 0.92])
        plt.savefig(os.path.join(plot_dir, f"{state}_error_comparison.png"), dpi=300)
        plt.close()


In [28]:
def create_fidelity_plots(experiments):
    """Create fidelity visualizations with finer grids and violin plots"""
    print("Creating improved state fidelity visualizations...")
    
    scatter_dir = os.path.join(PLOTS_DIR, "improved_fidelity_scatters")
    violin_dir = os.path.join(PLOTS_DIR, "state_fidelity_violins")
    os.makedirs(scatter_dir, exist_ok=True)
    os.makedirs(violin_dir, exist_ok=True)
    
    states = sorted({exp['initial_state'] for exp in experiments.values()})
    
    # PART 1: Create individual state scatter plots with finer grids
    print("  Creating individual state scatter plots...")
    for state in states:
        fig, ax = plt.subplots(figsize=(12, 8))
        
        state_data = [exp for exp in experiments.values() if exp['initial_state'] == state]
        if not state_data:
            continue
        
        # Setup color mapping
        norm = mpl.colors.Normalize(vmin=0, vmax=1)
        cmap = plt.cm.viridis
        
        # Plot with improved aesthetics
        for exp in state_data:
            sc = ax.scatter(
                x=exp['data'].index,
                y=exp['data']['fidelity_with_qec'],
                c=exp['data']['fidelity_with_qec'],
                cmap=cmap,
                norm=norm,
                alpha=0.7,
                edgecolor='w',
                linewidth=0.3,
                s=20  # Smaller points
            )
        
        # Add finer grid
        ax.grid(which='major', color='#DDDDDD', linewidth=0.8, alpha=0.7)
        ax.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5, alpha=0.5)
        ax.set_axisbelow(True)
        
        # Add minor ticks
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        
        # Format plot
        ax.set_title(f"Fidelity Distribution for {state} State", pad=20)
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fidelity")
        ax.set_ylim(0, 1.05)
        
        # Add colorbar
        cbar = fig.colorbar(sc, ax=ax)
        cbar.set_label("Fidelity Value")
        
        plt.tight_layout()
        plt.savefig(os.path.join(scatter_dir, f"{state}_fidelity_scatter.png"), dpi=300)
        plt.close()
    
    # PART 2: Create the all-states violin plot
    print("  Creating combined violin plot of all states...")
    fig, ax = plt.subplots(figsize=(14, 10))
    
    all_violins = []
    labels = []
    positions = []
    
    for i, state in enumerate(states):
        state_data = [exp['data']['fidelity_with_qec'] for exp in experiments.values() 
                     if exp['initial_state'] == state]
        
        if state_data:
            all_violins.append(np.concatenate(state_data))
            labels.append(state)
            positions.append(i+1)
    
    if all_violins:
        parts = ax.violinplot(all_violins, positions=positions, showmeans=True, showextrema=True)
        
        # Style violins
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(plt.cm.tab10(i))
            pc.set_edgecolor('black')
            pc.set_alpha(0.7)
        
        parts['cmeans'].set_color('black')
        parts['cmeans'].set_linewidth(1.5)
        
        # Add median values
        for i, data in enumerate(all_violins):
            ax.text(positions[i], np.median(data) + 0.02, 
                   f"{np.median(data):.3f}", 
                   ha='center', va='bottom', fontsize=10)
        
        ax.set_title("Fidelity Distribution Comparison Across Qubit States", pad=20, fontsize=16)
        ax.set_ylabel("Fidelity", fontsize=14)
        ax.set_xticks(positions)
        ax.set_xticklabels(labels)
        ax.set_ylim(0, 1.05)
        
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.grid(True, which='both', axis='y', alpha=0.5, linestyle=':')
        
        plt.tight_layout()
        plt.savefig(os.path.join(violin_dir, "all_states_fidelity_violin.png"), dpi=300)
        plt.close()
    
    print("Fidelity visualizations complete!")


In [38]:
def create_qec_analysis(experiments):
    """Create properly aggregated QEC method comparison plots"""
    print("Creating properly aggregated QEC method analysis...")
    
    plot_dir = os.path.join(PLOTS_DIR, "better_qec_analysis")
    os.makedirs(plot_dir, exist_ok=True)
    
    # Configure beautiful typography with LaTeX-style formatting
    plt.rcParams.update({
        'text.usetex': False,  # Set to True if LaTeX is installed
        'mathtext.fontset': 'stix',
        'font.family': 'STIXGeneral',
        'font.weight': 'normal',
        'axes.labelweight': 'bold',
        'axes.titleweight': 'bold',
        'axes.titlesize': 16,
        'axes.labelsize': 14
    })
    
    # Get unique states and create proper QEC method groups
    states = sorted({exp['initial_state'] for exp in experiments.values()})
    
    # Define proper state display names with LaTeX notation
    state_display = {
        '0': r'$|0\rangle$',
        '1': r'$|1\rangle$',
        '+': r'$|+\rangle$',
        '-': r'$|-\rangle$',
        '+i': r'$|+i\rangle$',
        '-i': r'$|-i\rangle$',
        'random': 'Random'
    }
    
    # Create true QEC method categories (not parameter-dependent)
    qec_categories = ['three_qubit_bit_flip', 'three_qubit_phase_flip', 
                      'five_qubit', 'shor_nine', 'steane_seven', 'none']
    
    # Professional naming and colors for consistency
    method_display = {
        'three_qubit_bit_flip': '3-Qubit Bit-Flip',
        'three_qubit_phase_flip': '3-Qubit Phase-Flip',
        'five_qubit': '5-Qubit Perfect',
        'shor_nine': 'Shor 9-Qubit',
        'steane_seven': 'Steane 7-Qubit',
        'none': 'No QEC'
    }
    
    method_colors = {
        'three_qubit_bit_flip': '#3498db',
        'three_qubit_phase_flip': '#2ecc71',
        'five_qubit': '#e74c3c',
        'shor_nine': '#9b59b6',
        'steane_seven': '#f39c12',
        'none': '#7f8c8d'
    }
    
    # Process each state
    for state in states:
        plt.figure(figsize=(12, 8))
        
        # Collect all fidelity data per QEC method (aggregated across ALL parameters)
        method_fidelities = {}
        
        for method in qec_categories:
            if method == 'none':
                # Get no-QEC baseline data across all experiments
                method_data = np.concatenate([
                    exp['data']['fidelity_without_qec'].values 
                    for exp in experiments.values()
                    if exp['initial_state'] == state
                ])
            else:
                # Aggregate data across all parameters for this method
                method_data = np.concatenate([
                    exp['data']['fidelity_with_qec'].values 
                    for exp in experiments.values()
                    if exp['initial_state'] == state and exp['qec_method'] == method
                ] or [np.array([])])  # Handle empty case
            
            if len(method_data) > 0:
                method_fidelities[method] = method_data
        
        # Prepare for plotting
        methods = list(method_fidelities.keys())
        positions = range(1, len(methods) + 1)
        
        # Create violin plots for distribution visualization
        vparts = plt.violinplot(
            [method_fidelities[m] for m in methods],
            positions=positions,
            showmeans=False,
            showextrema=False
        )
        
        # Style violin plots
        for i, pc in enumerate(vparts['bodies']):
            pc.set_facecolor(method_colors.get(methods[i], 'gray'))
            pc.set_edgecolor('none')
            pc.set_alpha(0.3)
        
        # Add boxplots for statistical summary
        bparts = plt.boxplot(
            [method_fidelities[m] for m in methods],
            positions=positions,
            patch_artist=True,
            widths=0.5,
            showfliers=False
        )
        
        # Style boxplots
        for i, box in enumerate(bparts['boxes']):
            box.set_facecolor(method_colors.get(methods[i], 'gray'))
            box.set_alpha(0.7)
            box.set_linewidth(1.5)
        
        # Add mean markers and text
        for i, method in enumerate(methods):
            mean_val = np.mean(method_fidelities[method])
            plt.scatter(positions[i], mean_val, color='white', s=30, 
                       zorder=3, edgecolor='black')
            plt.text(positions[i], mean_val + 0.03, f"{mean_val:.3f}", 
                    ha='center', fontsize=11, fontweight='bold')
        
        # Add a reference line at perfect fidelity
        plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7)
        
        # Format plot with proper spacing and labels
        plt.xlabel('QEC Method', fontweight='bold')
        plt.ylabel('Fidelity', fontweight='bold')
        plt.title(f'QEC Method Performance for {state_display.get(state, state)} State', 
                 fontsize=18, pad=20)
        
        # Set readable x-tick labels with proper method names
        plt.xticks(positions, [method_display.get(m, m) for m in methods], rotation=25)
        plt.yticks(np.arange(0, 1.1, 0.1))
        
        # Add grid lines
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.ylim(0, 1.05)
        
        # Add finer grid for better readability
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(5))
        plt.grid(which='minor', axis='y', linestyle=':', alpha=0.3)
        
        # Adjust layout and save
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dir, f"{state}_qec_comparison.png"), dpi=300)
        plt.close()
    
    print("Created properly aggregated QEC method comparison plots!")


In [35]:
def run_complete_analysis():
    """
    Run the complete quantum error correction analysis pipeline.
    """
    print("Starting comprehensive analysis...")
    
    # Load all experiment data
    experiments = load_experiment_data()
    
    # Run analyses
    # print("\nPerforming state-wise analysis...")
    # state_summaries = analyze_by_initial_state(experiments)
    
    # print("\nPerforming QEC method analysis...")
    # qec_summaries = analyze_by_qec_method(experiments)
    
    # print("\nPerforming error type analysis...")
    # error_summaries = generate_error_type_analysis(experiments)
    
    # print("\nGenerating visualizations...")
    # create_scatter_plots(experiments)

    # Run visualizations
    print("\nGenerating improved visualizations...")
    #create_state_error_plots(experiments)
    #create_fidelity_plots(experiments)
    create_qec_analysis(experiments)
    
    print("Analysis complete!")
    print(f"Results saved to {PLOTS_DIR} directory")
    
    # print("\nGenerating final report...")
    # report = generate_comprehensive_report(experiments, state_summaries, qec_summaries, error_summaries)
    
    print("\nAnalysis complete!")
    print(f"Results saved to {PLOTS_DIR} directory")
    #return experiments, state_summaries, qec_summaries, error_summaries, report
    return experiments

In [39]:
if __name__ == "__main__":
    run_complete_analysis()

Starting comprehensive analysis...
Loading experiment data...
Processing bit_phase_flip error type...
Processing bit_flip error type...
Processing gen_amplitude_damping error type...
Processing mixed_xz error type...
Processing phase_damping error type...
Processing phase_flip error type...
Processing amplitude_damping error type...
Processing coherent_overrotation error type...
Processing depolarizing error type...
Loaded 1638 experiments

Generating improved visualizations...
Creating properly aggregated QEC method analysis...
Created properly aggregated QEC method comparison plots!
Analysis complete!
Results saved to plot_results directory

Analysis complete!
Results saved to plot_results directory
